# Run the model

In [1]:
DEVICE_NUM = 3

In [2]:
from PIL import Image
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

# The line `model_id = "/home/t-sijoshi/multimodal-data-gen/output/spatial_map_500_canonical_vt_ft"` is assigning a specific path to a directory or file to the variable `model_id`. This path likely points to a pre-trained model or model checkpoint stored on the local file system. The code may be attempting to load a pre-trained model from the specified directory for further processing or fine-tuning.
model_id = "/home/t-sijoshi/multimodal-data-gen/output/spatial_map_500_canonical_vt_ft"
model_id  = "microsoft/Phi-3-vision-128k-instruct"


model = AutoModelForCausalLM.from_pretrained(model_id, device_map=f"cuda:{DEVICE_NUM}", trust_remote_code=True, torch_dtype=torch.float16, _attn_implementation="eager")


processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/home/t-sijoshi/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.85s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from IPython.display import display

query = " The user asks a multiple choice question on spatial reasoning given a figure.\n \
                Please rate the accuracy of the AI Answer compared to the Correct answer.\n \
                The assistant receives a binary score for the answer, of 0 or 1, where 1 means the answer is correct, 0 means incorrect answer.\n \
                Please output only 0 or 1 indicating whether the answer from the AI is correct or not. \
                Options: \
                A A black person living in Canada \
                B People whose ancestors came to Canada through the slave trade \
                C A white person who originated in Africa and now lives in Canada \
                D Any of the above \
                Correct: D \
                AI Answer: B"

# image = Image.open("/home/t-sijoshi/multimodal-data-gen/downloaded_datasets/spatial_map_canonical_500/images/map_2000.png")#
# display(image.resize((256,256)))

messages = [
    {"role": "human", "content": f"{query}"},
]

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)
inputs = processor(prompt,  return_tensors="pt").to(f"cuda:{DEVICE_NUM}")
generation_args = {
    "max_new_tokens": 10,
    "temperature": 0.0,
    "do_sample": False,
}

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

# remove input tokens
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

display(response)

<|human|>
 The user asks a multiple choice question on spatial reasoning given a figure.
                 Please rate the accuracy of the AI Answer compared to the Correct answer.
                 The assistant receives a binary score for the answer, of 0 or 1, where 1 means the answer is correct, 0 means incorrect answer.
                 Please output only 0 or 1 indicating whether the answer from the AI is correct or not.                 Options:                 A A black person living in Canada                 B People whose ancestors came to Canada through the slave trade                 C A white person who originated in Africa and now lives in Canada                 D Any of the above                 Correct: D                 AI Answer: B<|end|>
<|assistant|>



'0'

In [4]:
from IPython.display import display
import json 
from tqdm import tqdm 
import jsonlines

num_total = 0
num_correct = 0
with open("/home/t-sijoshi/multimodal-data-gen/downloaded_datasets/spatial_map_visual_only_v0/data.json", "r") as file:
  correct_reader = json.load(file)
  with jsonlines.open("/home/t-sijoshi/multimodal-data-gen/results/spatial_map_v0_2024_07_16_00:53:59.jsonl") as ai_reader:
    pbar = tqdm(zip(correct_reader, ai_reader), total=len(correct_reader))
    for datum, result in pbar:
      correct = datum["conversations"][1]["value"]
      num_total += len(correct)
      ai_answer = result["response"]
      query = f"We would like to request your feedback on the performance of an AI assistant.\n \
          The user asks the question on spatial reasoning given a figure.\n \
          Please rate the accuracy of the AI assistant answer compared to the Correct answer.\n \
          The assistant receives an binary score for each answer, of 0 or 1, where 1 means the answer is correct, 0 means incorrect answer.\n \
          The total score is the sum of the binary scores for each answer \n \
          Answers are denoted by A1, A2 and so on.\n \
          Please first output a single line containing only a value indicating the total scores for Assistant\n \
          Correct: {str(correct)}\n \
          AI Answer: {str(ai_answer)}"

      messages = [
          {"role": "user", "content": f"{query}"},
      ]

      prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
      inputs = processor(prompt, return_tensors="pt").to(f"cuda:{DEVICE_NUM}")
      generation_args = {
          "max_new_tokens": 1000,
          #"temperature": 0.0,
          "do_sample": False,
      }

      generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

      # remove input tokens
      generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
      response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
      num_correct += int(response)
      
      pbar.set_postfix({"acc": num_correct / num_total})

  

FileNotFoundError: [Errno 2] No such file or directory: '/home/t-sijoshi/multimodal-data-gen/results/spatial_map_v0_2024_07_16_00:53:59.jsonl'

In [8]:
print(num_total)

300
